In [1]:
import torch
from datasets import load_dataset
from datasets import load_from_disk
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [100]:
#定义数据集
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, split):
        # dataset = load_dataset(path='lansinuote/ChnSentiCorp', split=split)
        dataset = load_from_disk('./data/ChnSentiCorp')['%s' % split]
        
        def f(data):
            return len(data['text']) > 40

        self.dataset = dataset.filter(f)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset['text'][i]
        
        # print('原始数据: ', text)
        # label = self.dataset['label'][i]
        #切分一句话为前半句和后半句
        sentence1 = text[:20]
        sentence2 = text[20: ]
        label = 0

        # 有一半的概率把后半句替换为一句无关的话
        if random.randint(0, 1) == 0:
            j = random.randint(0, len(self.dataset) - 1)
            # sentence2 = self.dataset[j]['text'][20:40]
            sentence2 = self.dataset['text'][j][20:]
            label = 1

        return sentence1, sentence2, label

# 加载
dataset = Dataset('train')

Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/train/cache-248dcc552a1bfcb6.arrow


In [88]:
for i in range(2):
    print(dataset[i])
    print("")

('选择珠江花园的原因就是方便，有电动扶梯直', '接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般', 0)

('15.4寸笔记本的键盘确实爽，基本跟台式', '机差不多了，蛮喜欢数字小键盘，输数字特方便，样子也很美观，做工也相当不错', 0)



In [101]:
# 测试拼接的数据

sentence1, sentence2, label = dataset[3]

len(dataset), sentence1, sentence2, label

(8001,
 '今天才知道这书还有第6卷,真有点郁闷:为',
 '什么同一套书有两种版本呢?当当网是不是该跟出版社商量商量,单独出个第6卷,让我们的孩子不会有所遗憾。',
 0)

In [102]:
from transformers import BertTokenizer
from transformers import BertModel


# 加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')

print(token)


# 加载预训练模型
pretrained = BertModel.from_pretrained('bert-base-chinese').to(device)

BertTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [103]:
def collate_fn(data):
    sents = [i[:2] for i in data]  # 句子一, 句子二
    labels = [i[2] for i in data]

    # print('sents: ', sents)
    
    #编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=300,
                                   return_tensors='pt',
                                   return_length=True,
                                   add_special_tokens=True)

    # input_ids: 编码之后的数字
    # attention_mask: 是补零的位置是0,其他位置是1
    # token_type_ids: 第一个句子和特殊符号的位置是0,第二个句子的位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']
    labels = torch.LongTensor(labels)

    #print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels


## 数据加载器

In [104]:

loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=8,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)



for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
    print(len(loader))
    print(token.decode(input_ids[0]))
    input_ids.shape, attention_mask.shape, token_type_ids.shape, labels
    break

1000
[CLS] 我 是 在 图 书 馆 发 现 这 套 书 的 ， 看 了 第 1 集 后 就 [SEP] 以 前 货 到 付 款 都 没 这 么 久 ， 真 是 的 ， 收 了 钱 就 不 紧 张 了 ， 烦 烦 烦 ！ ！ ！ ！ ！ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [93]:
#不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)

In [105]:
# 定义下游任务模型
class Model(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                             attention_mask=attention_mask,
                             token_type_ids=token_type_ids)
            pass
        out = self.fc(out.last_hidden_state[:, 0])
        out = out.softmax(dim=1)
        return out

In [110]:
model = Model()
model.load_state_dict(torch.load('chinese_infer_mission_2023_4_10.pt'))
# 加载模型

# 模型转移到GPU上
model.to(device)

Model(
  (fc): Linear(in_features=768, out_features=2, bias=True)
)

In [111]:
# 句子测试封装


def str_felling_detect(model, str_sents):
    
    out = token.encode_plus(str_sents,
                            truncation=True,
                            padding='max_length',
                            max_length=45,
                            return_tensors='pt',
                            return_length=True,
                            add_special_tokens=True)
    # print(out)
    input_ids = out['input_ids'].to(device)
    attention_mask = out['attention_mask'].to(device)
    token_type_ids = out['token_type_ids'].to(device)
    
    out_test = model(input_ids=input_ids,
                     attention_mask=attention_mask,                 
                     token_type_ids=token_type_ids
                )
    
    t1 = out_test[0][0].item()
    t2 = out_test[0][1].item()
    
    print('相关性: %.4f \t 不相关性: %.4f' % (t1, t2))
    pred = out_test.argmax(dim=1).cpu().item()
    
    # print(pred)
    if pred == 1:
        print('%s 句子不相关' % pred)
    elif pred == 0:
        print('%s 相关' % pred)
    else:
        print('%s 未知' % pred)

    return pred

In [113]:

# str_sents = '他说明天要去钓鱼, 在中山公园. 您的快递到了'
str_sents = '您好, 您的快递到了'


predRes = str_felling_detect(model, str_sents)

相关性: 0.7952 	 不相关性: 0.2048
0 相关


In [117]:


sum_total = 0
sum_acc_count = 0

for i, str_sents in enumerate(Dataset('test')):
    
    input_str = str_sents[0] + str_sents[1]
    label = str_sents[2]
    
    print(i, input_str, label)
    
    predRes = str_felling_detect(model, input_str)
    
    if predRes == label:
        sum_acc_count += 1
    
    
    sum_total += 1
    print("")
    
    
    if i >= 50:
        break

print('acc: %.4f' % (sum_acc_count / sum_total))

Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/test/cache-c06d6d85e943fbe5.arrow


0 怀着十分激动的心情放映，可是看着看着发现的企业近一些。房间总是能听得很响的水声。希望酒店能降低价格。目前的设施同安徽省内其它酒店相比只值250元不到啊 1
相关性: 0.3293 	 不相关性: 0.6707
1 句子不相关

1 还稍微重了点，可能是硬盘大的原故，还要再Afee）是在第一次启动时决定是否安装。不像很多品牌的笔记本，一上来一股脑儿给你装上。而且实际使用后发现它的很多自带的实用软件非常出色，非常人性化！回家仔细使用后发现键盘手感果然如大家所说非常出色。另外第一次使用小红点就爱上它了--感觉比触摸板好多了，难怪它会成为ThinkPad基因之一机器虽然有点重，但是感觉很扎实，比某些“娱乐本”扎实很多。 1
相关性: 0.2750 	 不相关性: 0.7250
1 句子不相关

2 不错，作者的观点很颠覆目前中国父母的教育方式，其实古人们对于教育已经有了很系统的体系了，可是现在的父母以及祖父母们更多的娇惯纵容孩子，放眼看去自私的孩子是大多数，父母觉得自己的孩子在外面只要不吃亏就是好事，完全把古人几千年总结的教育古训抛在的九霄云外。所以推荐准妈妈们可以在等待宝宝降临的时候，好好学习一下，怎么把孩子教育成一个有爱心、有责任心、宽容、大度的人。 0
相关性: 0.2644 	 不相关性: 0.7356
1 句子不相关

3 有了第一本书的铺垫，读第二本的时候开始进入状态。基本上第二本就围绕主角们的能力训练展开，故事的主要发生场地设置在美洲的亚马逊丛林。心里一直疑惑这和西藏有什么关系，不过大概看完全书才能知道内里的线索。其中描述了很多热带雨林中特有的神秘动植物以及一些生存技巧和常识，受益匪浅。能够想像出要写这样一部书，融合这样许多的知识，作者需要花费多少心血来搜集和整理并成文。 0
相关性: 0.4919 	 不相关性: 0.5081
1 句子不相关

4 前台接待太差，酒店有A B楼之分，本人check－in后，前台未告诉B楼在何处，并且B楼无明显指示；房间太小，根本不像4星级设施，下次不会再选择入住此店啦。 0
相关性: 0.3153 	 不相关性: 0.6847
1 句子不相关

5 1. 白色的，很漂亮，做工还可以； 2. 网上的软件资源非常丰富，这是我买它的最主要原因； 3. 电池不错，昨天从下午两点到晚上十点还有25分钟的剩余时间（关闭摄像头，无线和

In [78]:
# 保存模型

# 保存

model_save_path = 'chinese_infer_mission_2023_4_10.pt'
# torch.save(model.state_dict(),  model_save_path)  # 推荐的文件后缀名是pt或pth